<a href="https://colab.research.google.com/github/PBHPBH/Machine_Learning/blob/NLP/4_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install JPype1
!pip install rhinoMorph

In [ ]:
cd /content/drive/MyDrive/pytest

In [ ]:
def read_data(filename, encoding='cp949'): # 읽기 함수 정의
    with open(filename, 'r', encoding=encoding) as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # txt 파일의 헤더(id document label)는 제외하기
    return data
def write_data(data, filename, encoding='cp949'): # 쓰기 함수도 정의
    with open(filename, 'w', encoding=encoding) as f:
        f.write(data)
#data = read_data('/content/gdrive/My Drive/pytest/ratings_small.txt', encoding='cp949')
data = read_data('ratings.txt', encoding='cp949') #(긍정 10만, 부정 10만)

In [ ]:
ls

In [ ]:
import rhinoMorph
rn = rhinoMorph.startRhino()

morphed_data = ''
for data_each in data:
    morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[1],
        pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
    joined_data_each = ' '.join(morphed_data_each) # 문자열을 하나로 연결
    if joined_data_each: # 내용이 있는 경우만 저장하게 함
        morphed_data += data_each[0]+"\t"+joined_data_each+"\t"+data_each[2]+"\n"

# 형태소 분석된 파일 저장
write_data(morphed_data, 'ratings_morphed.txt', encoding='cp949')

In [ ]:
data = read_data('ratings_morphed.txt' , encoding='cp949')
print(len(data))
print(len(data[0])) 
print(data[0])

In [ ]:
data_id = [line[0] for line in data] # 데이터 id
data_text = [line[1] for line in data] # 데이터 본문
data_senti = [line[2] for line in data] # 데이터 긍부정 부분
positive = read_data('positive.txt') # 긍정 감정사전 읽기
negative = read_data('negative.txt') # 부정 감정사전 읽기
print('positive:', positive)
print('negative:', negative)
pos_found = []
neg_found = []

In [ ]:
def cntWordInLine(data, senti):
    senti_found = []
    for onedata in data:
        oneline_word = onedata.split(' ') # 한 줄의 데이터를 공백 단위로 분리하여 리스트로 저장
        senti_temp = 0 # 그 줄에서 발견된 감정단어의 수를 담는 변수
        for sentiword in senti: # 감정사전의 어휘
            if sentiword[0] in oneline_word: # sentiword[0] 하여 리스트 원소를 문자열로 추출
                senti_temp += 1 # 현재의 감정단어와 일치하면 숫자를 하나 올려 줌 (중복X)
        senti_found.append(senti_temp) # 현재의 줄에서 찾은 감성단어의 숫자를 해당 위치에 저장
    return senti_found

data_senti_poscnt = cntWordInLine(data_text, positive) # 발견된 긍정 단어의 숫자 파악
data_senti_negcnt = cntWordInLine(data_text, negative) # 발견된 부정 단어의 숫자 파악

print(data_senti_poscnt[:20])
print(data_senti_negcnt[:20])

[5, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0]
[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]


In [ ]:
#감정점수 계산
# pandas 데이터프레임으로 저장
import pandas as pd
newdata = pd.DataFrame({'id':data_id, 'text':data_text,'original':data_senti, 'pos':data_senti_poscnt, 'neg':data_senti_negcnt})

senti_score = newdata['pos'] - newdata['neg']#긍정 개수에서 부정개수를 뺌
newdata['senti_score'] = senti_score# 그 수를 senti_score 컬럼에 저장

newdata.loc[newdata.senti_score > 0, 'new']=1#새로운 긍부정 기호
newdata.loc[newdata.senti_score <= 0, 'new']=0#새로운 긍부정 기호

#처음에 기록된 긍부정과 새로 계산된 긍부정이 같은지 여부를 matched컬럼에 저장
#original 컬럼은 문자로 되어 있으므로 숫자로 변환 뒤 비교
newdata.loc[pd.to_numeric(newdata.original) == newdata.new, 'matched'] = 'True'
newdata.loc[pd.to_numeric(newdata.original) != newdata.new, 'matched'] = 'False'

In [ ]:
#원점수 와 비교 및 저장
score = newdata.matched.str.count('True').sum()/(newdata.matched.str.count('True').sum()+newdata.matched.str.count('false').sum())* 100
print(score)

newdata.to_csv('newfile.csv', sep=',', encoding='cp949', index=False)#CSV저장
newdata.to_csv('newfile2.txt', sep='\t',encoding='cp949', index=False)#또는 txt저장

100.0


In [ ]:
#시그모이드 점수 계산
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))
newdata['sigmoid'] = newdata.senti_score.apply(sigmoid)


In [ ]:
#결과 확인
newdata.head()

,id,text,original,pos,neg,senti_score,new,matched,sigmoid
0,8132799,디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라...,1,5,1,4,1.0,True,0.982014
1,4655635,폴리스스토리 시리즈 뉴 없 최고,1,1,0,1,1.0,True,0.731059
2,9251303,와 연기 진짜 쩔 지루 생각하 몰입 그래 이런 진짜 영화,1,0,1,-1,0.0,False,0.268941
3,10067386,안개 자욱 하 밤하늘 뜨 초승달 같 영화,1,0,0,0,0.0,False,0.500000
4,2190435,사랑 해보 사람 처음 끝 웃 있 영화,1,2,0,2,1.0,True,0.880797


In [ ]:
################################################################################################################################################################################################

In [ ]:
ls

네이버뉴스-생활문화_다중/  kor-eng/      pytest_position.png  test.csv
aclImdb_v1_small/          negative.txt  ratings_morphed.txt  김소월시.txt
alice.png                  newfile2.txt  ratings_small.txt    윤동주시.txt
fra-eng/                   newfile.csv   ratings.txt          wiki_test.txt
iris.csv                   positive.txt  similarity/          word2vec/


In [ ]:
def read_data(filename, encoding='cp949'): # 읽기 함수 정의
    with open(filename, 'r', encoding=encoding) as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # txt 파일의 헤더(id document label)는 제외하기
    return data
def write_data(data, filename, encoding='cp949'): # 쓰기 함수 정의
    with open(filename, 'w', encoding=encoding) as f:
        f.write(data)

data = read_data('ratings_morphed.txt' , encoding='cp949')
print(len(data))
print(len(data[0])) 
print(data[0])

197513
3
['8132799', '디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사', '1']


In [ ]:
#훈련데이터와 테스트데이터 분리(자동)
data_text = [line[1] for line in data]#데이터 본문
data_senti = [line[2] for line in data] # 데이터 긍부정 부분

from sklearn.model_selection import train_test_split # 본문과 라벨을 각각 분리
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify=data_senti)

# Counter 클래스를 이용해 각 분류가 훈련데이터와 테스트데이터에 같은 비율로 들어갔는지 확인해 본다
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)

train_data_senti_freq: Counter({'1': 74097, '0': 74037})
test_data_senti_freq: Counter({'1': 24699, '0': 24680})


In [ ]:
#훈련데이터와 테스트데이터 분리(수동)
import random
random.shuffle(data) # 랜덤하게 섞는다

data_70 = int(len(data)*0.7) # 전체 데이터 크기의 70% 숫자를 찾는다
train_data = data[:data_70] # 앞에서 70% 부분을 잘라 훈련데이터로
test_data = data[data_70:] # 그 뒷부분을 테스트데이터로

print('train data length:', len(train_data)) # 138079
print('test data length:', len(test_data)) # 59178

# 훈련데이터 요소 분리
train_data_text = [line[1] for line in train_data] # 훈련데이터 본문
train_data_senti = [line[2] for line in train_data] # 훈련데이터 긍부정 부분

# 테스트데이터 요소 분리
test_data_text = [line[1] for line in test_data] # 테스트데이터 본문
test_data_senti = [line[2] for line in test_data] # 테스트데이터 긍부정 부분

train data length: 138259
test data length: 59254


In [ ]:
#행렬형태로 변환
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5).fit(train_data_text) # 최소 문서 빈도 5이상의 단어만 대상
X_train = vect.transform(train_data_text) # 행렬 생성
print("X_train:\n", repr(X_train)) 

X_train:
 <138259x11298 sparse matrix of type '<class 'numpy.int64'>'
	with 766107 stored elements in Compressed Sparse Row format>


In [ ]:
#행렬내용 관찰
feature_names = vect.get_feature_names()
print("특성 개수:", len(feature_names))
print("처음 20개 특성:\m", feature_names[:20])
print("3000~5000까지의 특성:\n", feature_names[3000:5000])

특성 개수: 11298
처음 20개 특성:\m ['10점', '1빠', 'ai', 'cgv', 'ebs', 'kbs', 'la', 'mb', 'mbc', 'naver', 'ok', 'sbs', 'sns', 'tv', 'ㄴㄴ', 'ㄷㄷ', 'ㅂㄷㅂㄷ', 'ㅂㅅ', 'ㅅㅂ', 'ㅇㅇ']
3000~5000까지의 특성:
 ['마찬가지', '마초', '마치', '마침', '마침내', '마케팅', '마크', '마키', '마트', '마틴', '마틴스', '마틸다', '마피아', '막가', '막가파', '막걸리', '막내', '막대', '막되', '막막', '막말', '막무가내', '막바지', '막상', '막스', '막심', '막연', '막장', '막지', '막판', '막히', '만가', '만개', '만그', '만끽', '만나', '만날', '만남', '만년', '만능', '만담', '만두', '만들', '만듦새', '만땅', '만만', '만명', '만발', '만보', '만사', '만세', '만약', '만인', '만점', '만족', '만족감', '만주', '만지', '만찬', '만하', '만해', '만행', '만화', '만화가', '만화영화', '만화책', '만회', '많이', '말기', '말길', '말년', '말랑말랑', '말로', '말론', '말리', '말세', '말씀', '말아먹', '말없이', '말입', '말자', '말장난', '말종', '말초신경', '말코비치', '말콤', '말타', '말투', '말하', '말하기', '말해', '맘껏', '맘대로', '맘마', '맘마미아', '맘속', '맙소사', '맛깔', '맛보', '맛없', '맛있', '맛집', '망가', '망가지', '망각', '망상', '망설이', '망설임', '망신', '망인', '망쳐놓', '망치', '망하', '맞먹', '맞물리', '맞서', '맞아떨어지', '맞이', '맞추', '맞춤', '맡기', '매개체', '매국노', '매기', '매끄럽', '매너', '매너리즘', '매년', '매니아', '매

In [ ]:
#머신러닝 알고리즘 적용
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
y_train = pd.Series(train_data_senti)#리스트 형태를 종속변수가 될수 있는 1차 배열(Series)로 만든다

lr = LogisticRegression(solver = 'liblinear')#모델생성
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#테스트 데이터 입력
X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)
print("테스트 데이터 점수:", lr.score(X_test, y_test))

테스트 데이터 점수: 0.8094643399601714


In [ ]:
# 형태소분석기 시작
import rhinoMorph
rn = rhinoMorph.startRhino()

new_input =  '오늘은 정말 재미있는 하루구나!'
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(
    rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input) # ['오늘', '정말', '재미있', '하루＇]를 한 개 문자열로 변환

inputdata.append(morphed_input) # 분석 결과를 리스트로 만들기
print('input data:', inputdata) # ['오늘 정말 재미있 하루']

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
input data: ['오늘 정말 재미있 하루']


In [ ]:
#1개 데이터 예측
X_input = vect.transform(inputdata) # 앞에서 만든 11445 컬럼의 행렬에 적용
result = lr.predict(X_input) # 0은 부정,1은 긍정

if result == "0" : # 문자열 형태로 출력된다
    print("부정적인 글입니다")
else:
    print("긍정적인 글입니다")

긍정적인 글입니다
